In [11]:
import warnings
warnings.filterwarnings("ignore") 

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import os, librosa
from tqdm import tqdm
import torchaudio
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import sklearn 
import pickle

In [12]:
with open('audio_data.pkl', 'rb') as f:
    audio_data_df = pickle.load(f)

In [13]:
audio_data_df[49][0].shape


(128, 126)

In [14]:
#split the data

from torch.utils.data import Dataset, DataLoader
from class_CNN import DatasetClass
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(audio_data_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

# call my dataclass 

train_data = DatasetClass(train_df)
val_data = DatasetClass(val_df)
test_data = DatasetClass(test_df)

#save test data to pickle file
# with open('test_data.pkl', 'wb') as f:
#     pickle.dump(test_data, f)

#call my dataloader 

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [15]:
print(train_data[0][0].shape)
for i, (X, y) in enumerate(train_loader):
    print(X.shape)
    print(y.shape)
    break



(128, 127)
torch.Size([32, 128, 127])
torch.Size([32])


In [16]:
# import model

from model_encoder import classification, CNN1D
import torch

model = classification()
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

classification(
  (cnn): CNN1D(
    (conv1): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (gelu): GELU(approximate='none')
  )
  (encoder_blocks): ModuleList(
    (0): EncoderBlock(
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mha): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (ffn): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=256, bias=True)
      )
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (classifier): Sequential(
    (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=256, out_features=10, bias=True)
  )
)

In [17]:
import wandb 
import datetime

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M_%S')
epochs = 10
batch_size = 32


wandb.init(
    project="week5",
    name=f"flick-{timestamp}",
    config={
    "epochs": epochs,
    "batch_size": batch_size,
    "learning_rate": 0.001
})

In [18]:

for epoch in range(20):
    model.train()
    total_loss = 0

    for padded_spec, label in train_loader:
        #print(padded_spec.shape)
        padded_spec = padded_spec.to(device)
        label = label.to(device)

        optimiser.zero_grad()

        output = model(padded_spec)

        loss = criterion(output, label)
        loss.backward()
        optimiser.step()
        
        total_loss += loss.item()
  
    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0

    with torch.no_grad():
        for padded_spec, label in val_loader:
            padded_spec = padded_spec.to(device)
            label = label.to(device)

            output = model(padded_spec)
            loss = criterion(output, label)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    wandb.log({"train_loss": avg_train_loss, "val_loss": avg_val_loss})
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

wandb.finish()
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    

Epoch 1, Train Loss: 1.9943, Val Loss: 1.7417
Epoch 2, Train Loss: 1.6360, Val Loss: 1.6966
Epoch 3, Train Loss: 1.4384, Val Loss: 1.5336
Epoch 4, Train Loss: 1.3043, Val Loss: 1.3220
Epoch 5, Train Loss: 1.1955, Val Loss: 1.3559
Epoch 6, Train Loss: 1.1367, Val Loss: 1.1085
Epoch 7, Train Loss: 1.0417, Val Loss: 1.2066
Epoch 8, Train Loss: 1.0134, Val Loss: 1.1837
Epoch 9, Train Loss: 0.9100, Val Loss: 0.9364
Epoch 10, Train Loss: 0.8808, Val Loss: 0.9345
Epoch 11, Train Loss: 0.8371, Val Loss: 0.8601
Epoch 12, Train Loss: 0.8146, Val Loss: 0.8964
Epoch 13, Train Loss: 0.7433, Val Loss: 0.8391
Epoch 14, Train Loss: 0.7270, Val Loss: 0.8394
Epoch 15, Train Loss: 0.6827, Val Loss: 0.7515
Epoch 16, Train Loss: 0.6398, Val Loss: 0.6834
Epoch 17, Train Loss: 0.6254, Val Loss: 0.7696
Epoch 18, Train Loss: 0.5836, Val Loss: 0.6141
Epoch 19, Train Loss: 0.5584, Val Loss: 0.6410


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 20, Train Loss: 0.5099, Val Loss: 0.6662


train_loss,█▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁
val_loss,██▇▅▆▄▅▅▃▃▃▃▂▂▂▁▂▁▁▁
train_loss,0.50987
val_loss,0.66617


In [19]:
torch.save(model.state_dict(), "encoder.pt")